The dataset is massive, so the idea is to extract only the relevant patent kinds and work from there. For that we need to connect to BigQuery and then use a typical SQL type query. 

In [43]:
import os
from google.cloud import bigquery
import pandas as pd
from IPython.display import display 
import db_dtypes

path = r"C:\Users\vedra\Dropbox (CLAPESUC)\Documentos\Pega\Research"

# Set the path to your JSON key
os.environ['GOOGLE_APPLICATION_CREDENTIALS'] = r"{}/Spotify Effect\Music_paper\Data\patents\BigQuery Keys/alert-palace-452300-q0-33798fdb6e91.json".format(path)

# Initialize BigQuery client
client = bigquery.Client()

Now we first gouge the size of the potential patents we are interested in. The initial codes are ``G10``, ``G11``, and ``H04``. This should include all music production innovations (``G10``), physical music storage (``G11``), and data transmission and compression (``H04``).

In [46]:
query = """
SELECT  t.publication_number,
        t.publication_date,
        FLOOR(t.publication_date/10000) AS publication_year,
        FLOOR((t.publication_date -FLOOR(t.publication_date/10000)*10000)/100) AS publication_month,
        t.country_code,
        t.title_localized[SAFE_OFFSET(0)].text as patent_title,
        t.abstract_localized[SAFE_OFFSET(0)].text as patent_abstract,
        ARRAY_LENGTH(t.citation) AS citation_count,
        ARRAY(SELECT LEFT(c.code, 3)
        FROM UNNEST(t.cpc) c
        WHERE LEFT(c.code, 3) = "G10"
        )[OFFSET(0)] AS cpc_code
FROM `patents-public-data.patents.publications` t
TABLESAMPLE SYSTEM(10 PERCENT)
WHERE EXISTS (
      SELECT 1
      FROM UNNEST(t.cpc) c
      WHERE LEFT(c.code, 3) = "G10"
)
"""

In [47]:
# Run the query
query_job = client.query(query)
results = query_job.to_dataframe()

Forbidden: 403 Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas; reason: quotaExceeded, location: unbilled.analysis, message: Quota exceeded: Your project exceeded quota for free query bytes scanned. For more information, see https://cloud.google.com/bigquery/docs/troubleshoot-quotas

Location: US
Job ID: 2a272403-6df0-492d-9315-87301334eeb9


In [38]:
output_path = r'C:\Users\vedra\Dropbox (CLAPESUC)\Documentos\Pega\Research\Spotify Effect\Music_paper\Data\patents'
results.to_csv(output_path + r"\G10_patents_10pct_sample.csv", index=False, quoting=1, escapechar="\\", encoding="utf-8")

       year cpc_code  total_patents
114  1950.0      G10            806
115  1951.0      G10            868
116  1952.0      G10            777
117  1953.0      G10            667
118  1954.0      G10            599
..      ...      ...            ...
185  2021.0      G10         271565
186  2022.0      G10         259829
187  2023.0      G10         255983
188  2024.0      G10         244520
189  2025.0      G10          19039

[76 rows x 3 columns]
